# Prototype 2 - Current Working Application ✔️

#### -Second attempt at facial recognition similarity checker.
#### -Uses multiple face encodings from the DB instead of images in local directory.
#### -Database integration ✔️
#### -Individual information stored and diplayed ✔️
#### -Real time webcam feedback with similarity percentage ✔️ 

## Shortcomings
### -Extremely slow real time feedback using the webcam
##### The code is slow because it has to retrieve the face encodings from the database as BLOB then convert to a numpy array, then perform various loops and calculations on thousands of numbers and it has to show the results on the webcam feed. 
###### One way to optimize the code is to use a pre-trained deep learning model for face recognition. Deepface is a Python library that covers Keras-based face recognition models and can recognize faces in real-time. According to experiments, VGG-Face is the fastest to build

###### Another option is to use a combination of deep learning and support vector machines (SVMs) for face recognition. This approach involves training a deep learning model to extract features from faces, then using an SVM to classify the faces based on those features
###### Try changing Database system to MySQL or MongoDB
###### - MySQL is a relational database management system that stores data in tables containing rows and columns, while MongoDB is a document-based NoSQL database that stores data in JSON-like documents. Face recognition requires storing multidimensional arrays and making calculations over them, which can be problematic for large data sets with millions of data points. MongoDB is well-suited for real-time data processing and big data operations, while MySQL is more popular for traditional web applications

###### This approach can achieve high accuracy and fast recognition times. Additionally, the code can be optimized by using multi-threading or multiprocessing to perform the face recognition and database queries in parallel, which can significantly reduce the execution time

###### recommended by perplexity.ai

## Query Database

In [2]:
import face_recognition, pickle, cv2, sqlite3, os, numpy as np

DATABASE_NAME = "face_data.db"
KNOWN_FACES_DIR = "Known_Faces"

def create_database():
    # Connect to the database or create it if it doesn't exist
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()

    # Create a table to store face data if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS face_data (
            id INTEGER PRIMARY KEY,
            email TEXT UNIQUE,
            name TEXT UNIQUE,
            age INTEGER,
            occupation TEXT,
            face_encoding BLOB,
            image_folder TEXT
        )
    ''')

    # Save the changes and close the connection
    conn.commit()
    conn.close()

def encode_image(image_path):
    image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(image)

    if len(face_encodings) > 0:
        return face_encodings[0]  # Only encode the first face found in the image
    else:
        return None

def check_existing_data(email, name, face_encoding, image_folder):
    try:
        # Connect to the database
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Check if the email, name, face encoding, or image path already exists in the database
        cursor.execute('''
            SELECT * FROM face_data WHERE email=? OR name=? OR face_encoding=? OR image_folder=?
        ''', (email, name, face_encoding, image_folder))

        existing_data = cursor.fetchone()

        # Close the connection
        conn.close()

        return existing_data

    except sqlite3.Error as e:
        print("Error while checking existing data:", e)
        return None

def insert_data(email, name, age, occupation, face_encoding, image_folder):
    try:
        # Check if the data already exists
        existing_data = check_existing_data(email, name, face_encoding, image_folder)
        if existing_data:
            print("Data already exists in the database")
            return

        # Connect to the database
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Insert data into the table
        cursor.execute('''
            INSERT INTO face_data (email, name, age, occupation, face_encoding, image_folder)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (email, name, age, occupation, face_encoding, image_folder))

        # Save the changes and close the connection
        conn.commit()
        conn.close()

        print("Data inserted successfully!")

    except sqlite3.Error as e:
        print("Error while inserting data:", e)

def main():
    create_database()

    # Sample data to insert (replace this with actual data and image paths)
    data = [
        {
            "email": "toluwaniojof@gmail.com",
            "name": "Ojo Toluwani",
            "age": 20,
            "occupation": "Mobile Developer",
            "image_folder": "Known_Faces/Tolu"
        },
        {
            "email": "john.doe@yahoo.com",
            "name": "Balogun Olamide",
            "age": 19,
            "occupation": "Data Scientist",
            "image_folder": "Known_Faces/Olamide"
        },
        {
            "email": "adisadavid@gmail.com",
            "name": "Adisa David",
            "age": 20,
            "occupation": "Full Stack Developer",
            "image_folder": "Known_Faces/David"
        },
        {
            "email": "afolabiife@gmail.com",
            "name": "Afolabi Oluwaife",
            "age": 19,
            "occupation": "Frontend Developer",
            "image_folder": "Known_Faces/Ife"
        },
        # {
        #     "email": "aalimah@gmail.com",
        #     "name": "Aalimah",
        #     "age": 10,
        #     "occupation": "Senior Prefect",
        #     "image_folder": "Known_Faces/Aalimah"
        # },
        {
            "email": "semilore@gmail.com",
            "name": "Oniyide Semilore",
            "age": 27,
            "occupation": "Software Intern",
            "image_folder": "Known_Faces/Semilore"
        }
    ]

    for item in data:
        email = item["email"]
        name = item["name"]
        age = item["age"]
        occupation = item["occupation"]
        image_folder = item["image_folder"]
        image_paths = [os.path.join(image_folder, image) for image in os.listdir(image_folder) if image.lower().endswith('.jpg')]
        face_encoding_list = []
        
        for image_path in image_paths:
            face_encoding = encode_image(image_path)
            if face_encoding is not None:
                face_encoding_list.append(face_encoding)
            else:
                print(f"Error: No face found in the image: {image_path}")
                # Delete any image with no face encoding
                os.remove(image_path)
        # Serializing the array, because it's too large, using pickle
        blob_face_encoding = pickle.dumps(np.array(face_encoding_list))
        insert_data(email, name, age, occupation, blob_face_encoding, image_path)

if __name__ == "__main__":
  main()

Data already exists in the database
Data already exists in the database
Data already exists in the database
Data already exists in the database
Data already exists in the database
Error: No face found in the image: Known_Faces/Semilore\0fb76587-5216-11ee-84d9-dc7196b8973b.jpg
Data inserted successfully!


## Test Face Recognition and DB info retrieval

In [3]:
import face_recognition, pickle, cv2, sqlite3, time, os, numpy as np

DATABASE_NAME = "face_data.db"

def load_known_face_encodings():
    known_face_encodings = {}
    try:
        # Connect to the database
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Load face encodings and other information from the database
        cursor.execute('''
            SELECT name, face_encoding FROM face_data
        ''')

        rows = cursor.fetchall()
        for name, face_encoding in rows:
            face_encoding = pickle.loads(face_encoding)
            known_face_encodings[name] = face_encoding

        # Close the connection
        conn.close()

    except sqlite3.Error as e:
        print("Error while loading known face encodings:", e)

    return known_face_encodings

def recognize_person(face_encoding, known_face_encodings):
    # Compare the face encoding with known faces (from the loaded array)
    for name, known_face_encoding in known_face_encodings.items():
        for face in known_face_encoding:
            matches = face_recognition.compare_faces([face], face_encoding, tolerance= 0.3)

            if any(matches):
                return name

    return None

def main():
    known_face_encodings = load_known_face_encodings()

    # Load Haar Cascade Classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Start webcam capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame.")
            break

        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]

            name = recognize_person(face_encoding, known_face_encodings)
            if name:
                # Calculate similarity percentage
                known_face_encoding_arr = known_face_encodings[name]
                for face in known_face_encoding_arr:
                    similarity_scores = face_recognition.face_distance([face], face_encoding)
                    similarity_percentage = (1 - similarity_scores[0]) * 100
                    if similarity_percentage > 70:
                        color = (90, 255, 0)
                        
                        try:
                            # Connect to the database
                            conn = sqlite3.connect(DATABASE_NAME)
                            cursor = conn.cursor()

                            # Fetch additional information from the database
                            cursor.execute('''
                                SELECT age, occupation, email FROM face_data WHERE name=?
                            ''', (name,))
                            row = cursor.fetchone()

                            # Close the connection
                            conn.close()

                            if row:
                                age, occupation, email = row
                                # Draw a rectangle around the face
                                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                                cv2.putText(frame, name, (x, y - 140), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                                cv2.putText(frame, str(age), (x, y -110), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                                cv2.putText(frame, str(occupation), (x, y - 80), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                                cv2.putText(frame, str(email), (x, y -50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                                cv2.putText(frame, str(round(similarity_percentage, 2)), (x, y -20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

                        except sqlite3.Error as e:
                            print("Error while fetching additional information:", e)
                            # time.sleep(10)
                            
                        break
                    else:
                        color = (0, 0, 255)
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        # cv2.putText(frame, "Unknown Face", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                        # cv2.putText(frame, str(similarity_percentage), (x, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                        

        # Show the frame
        cv2.imshow("Webcam", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

### Rough Snippet Testing Cell

In [4]:
# import pickle
image_folder = "Known_Faces/Olamide/"
image_paths = [os.path.join(image_folder, image) for image in os.listdir(image_folder) if image.lower().endswith('.jpg')]
img = cv2.imread(image_paths[0])
print(np.array(face_recognition.face_encodings(img)))

my_face_encoding = face_recognition.face_encodings(img, face_recognition.face_locations(img))
# print(my_face_encoding)

known_face_encodings = {}
# Connect to the database
conn = sqlite3.connect(DATABASE_NAME)
cursor = conn.cursor()
# Load face encodings and other information from the database
cursor.execute('''
    SELECT name, face_encoding FROM face_data
''')

rows = cursor.fetchall()
for name, face_encoding in rows:
    face_encoding = pickle.loads(face_encoding)
    known_face_encodings[name] = face_encoding
    
print(known_face_encodings)
# # for name, known_face_encoding in known_face_encodings.items():
# #     for face in known_face_encoding:
# #         # print([face])
# #     # print(f"{name}\n{known_face_encoding}")
# #     # break
    
# known_face_encoding_arr = known_face_encodings[name]
# # Close the connection
# conn.close()

# for f in known_face_encoding_arr:
#     similarity = face_recognition.face_distance(f, my_face_encoding)
#     print(similarity)
#     print(np.array(my_face_encoding).shape)


[[-0.14822902  0.12806346  0.07764941 -0.0110417   0.0339525  -0.09603757
   0.06363093 -0.11230733  0.09338085 -0.07368533  0.33839285 -0.00807128
  -0.15399936 -0.18805668  0.13684735  0.14158083 -0.10072909 -0.11208339
  -0.14312361 -0.15838525 -0.05827253  0.02193651  0.03035857  0.12359738
  -0.04729208 -0.28716436 -0.13595127 -0.12722483  0.13783737 -0.1239616
  -0.01900975  0.03006233 -0.21241911 -0.05140668 -0.04490493  0.02880214
   0.00911956 -0.06313883  0.21016355  0.06716155 -0.09693323 -0.02659038
  -0.0540265   0.29434437  0.13666478 -0.05917297  0.05195214  0.0207455
   0.05789187 -0.13186096  0.02811381  0.09414488  0.15014416  0.05453746
  -0.06758855 -0.14663108 -0.04804978 -0.00567514 -0.15493611  0.02225381
   0.02364139 -0.15147278 -0.16223153 -0.02642655  0.24232258  0.11515862
  -0.10349224 -0.18969154  0.19345325 -0.10281379  0.00571243  0.13879496
  -0.1718055  -0.08072777 -0.26517126  0.20442209  0.34387565  0.04926274
  -0.17876409 -0.01797874 -0.22544634  0